In [ ]:
import numpy as np
from ortools.linear_solver import pywraplp

# -------------------------------
# Problem Setup
# -------------------------------
num_nurses = 4
num_days = 3
num_shifts = 3
shifts = ['AM', 'PM', 'Night']

# Simulated logistic regression probabilities (scaled 0–10)
prob_matrix = np.array([
    [[0.9, 0.1, 0.2], [0.8, 0.2, 0.1], [0.9, 0.1, 0.2]],  # Nurse 0
    [[0.85, 0.1, 0.05], [0.7, 0.15, 0.2], [0.6, 0.25, 0.1]],  # Nurse 1
    [[0.1, 0.9, 0.2], [0.2, 0.85, 0.1], [0.1, 0.95, 0.2]],  # Nurse 2
    [[0.2, 0.15, 0.85], [0.1, 0.2, 0.8], [0.1, 0.25, 0.85]]  # Nurse 3
])

# Simulated prior schedule (baseline)
baseline_schedule = {
    (0, 0, 0): 1, (1, 0, 1): 1, (2, 0, 2): 1,
    (0, 1, 0): 1, (2, 1, 1): 1, (3, 1, 2): 1,
    (1, 2, 0): 1, (2, 2, 1): 1, (3, 2, 2): 1,
}

# -------------------------------
# Solver Initialization
# -------------------------------
solver = pywraplp.Solver.CreateSolver('SCIP')
x = {}

# Decision variables: x[nurse, day, shift]
for n in range(num_nurses):
    for d in range(num_days):
        for s in range(num_shifts):
            x[n, d, s] = solver.IntVar(0, 1, f'x[{n},{d},{s}]')

# -------------------------------
# Hard Constraints
# -------------------------------

# Constraint 1: Each shift per day must be assigned to 1 nurse
for d in range(num_days):
    for s in range(num_shifts):
        solver.Add(sum(x[n, d, s] for n in range(num_nurses)) == 1)

# Constraint 2: Each nurse can work at most 1 shift per day
for n in range(num_nurses):
    for d in range(num_days):
        solver.Add(sum(x[n, d, s] for s in range(num_shifts)) <= 1)

# -------------------------------
# Trust Region Constraint (Soft stability control)
# -------------------------------
max_changes = 2  # Allow only 2 changes from the prior schedule
diff_vars = []

for key, var in x.items():
    prev_value = baseline_schedule.get(key, 0)  # 1 or 0
    diff = solver.IntVar(0, 1, f'diff_{key}')
    diff_vars.append(diff)

    # Constraint: diff = 1 if current value ≠ baseline
    solver.Add(var - prev_value <= diff)
    solver.Add(prev_value - var <= diff)

# Limit number of changes allowed from baseline
solver.Add(solver.Sum(diff_vars) <= max_changes)

# -------------------------------
# Objective: Maximize total preference score
# -------------------------------
objective = solver.Objective()
for n in range(num_nurses):
    for d in range(num_days):
        for s in range(num_shifts):
            objective.SetCoefficient(x[n, d, s], prob_matrix[n][d][s])
objective.SetMaximization()

# -------------------------------
# Solve
# -------------------------------
status = solver.Solve()

# -------------------------------
# Output
# -------------------------------
if status == pywraplp.Solver.OPTIMAL:
    print(" Optimal assignment (with trust region):")
    total_changes = 0
    for d in range(num_days):
        print(f"\n🗓 Day {d+1}")
        for s in range(num_shifts):
            for n in range(num_nurses):
                if x[n, d, s].solution_value() == 1:
                    assigned = (n, d, s)
                    baseline_val = baseline_schedule.get(assigned, 0)
                    changed = "(changed)" if baseline_val != 1 else ""
                    if changed: total_changes += 1
                    print(f"  Shift {shifts[s]} → Nurse {n} {changed} (score: {prob_matrix[n][d][s]:.2f})")
    print(f"\n Total changes from baseline: {total_changes}")
else:
    print(" No optimal solution found.")


load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Alex\anaconda3\envs\nurse\lib\site-packages\ortools\.libs\ortools.dll...
✅ Optimal assignment (with trust region):

🗓 Day 1
  Shift AM → Nurse 0  (score: 0.90)
  Shift PM → Nurse 1  (score: 0.10)
  Shift Night → Nurse 3 (changed) (score: 0.85)

🗓 Day 2
  Shift AM → Nurse 0  (score: 0.80)
  Shift PM → Nurse 2  (score: 0.85)
  Shift Night → Nurse 3  (score: 0.80)

🗓 Day 3
  Shift AM → Nurse 1  (score: 0.60)
  Shift PM → Nurse 2  (score: 0.95)
  Shift Night 